## 如何在Ubuntu 18.04中修改和配置网卡地址和静态IP

### 1、 前言

- 用于超算中心的网络配置，超算中心有线上网需要备案和交网费，所以只能使用静态IP入网，不能使用默认的DHCP     
- 不要使用Ubuntu 18.04的网络图形话界面进行配置（**Settings -> Network -> Wired -> Connection Profile**)。因为那个图形界面NetworkManager做得很垃圾，配置不会生效，会疯掉的
- `Ubuntu 18.04`比`16.04`多加了`Netplan`网络管理工具，所以配置方式有所不同，会在下面进行详细的说明
- 最简单省钱的配置方法就是连接517里面MiaGroup的WiFi（台式机需要加装无线网卡），这样子只需要使用默认的DHCP就行了，即插即用

### 2、Ubuntu 16.04的配置方法

打开终端
``` bash
alt + ctrl + t
```
然后使用命令行修改配置文件interfaces：   
``` bash
$ sudo gedit /etc/network/interfaces
```
默认情况下，该配置文件中应该只有以下3行内容
``` python
# interfaces(5) file used by ifup(8) and ifdown(8)                              
auto lo
iface lo inet loopback
```
其中`#`表示注释，`lo`表示的是本机内部环路，具体含义我也不懂，是每台电脑都有的，`auto`表示开机即启动一下配置内容。 
这时我们需要做的的是加入以下几行的配置文件，对我们的有线网卡进行配置即可：
``` python
# interfaces(5) file used by ifup(8) and ifdown(8)                              
auto lo
iface lo inet loopback

# enp0s31f6是网卡的编号，一般为enp0，我这在主机（华硕Z370主板）上的则是奇怪的 enp0s31f6
auto enp0s31f6
# static表示设置为静态IP
iface enp0s31f6 inet static
# IP地址，按照申请的结果填写，一下的掩码、网关和dns也一样
address 172.18.166.×××
netmask 255.255.254.0
gateway 172.18.167.254
dns-nameservers 10.8.8.8
# 需要更改的网卡地址，也就是物理地址，也就是mac地址
hwaddress ether 8C:EC:4B:45:××:××
```

然后就根据上面的注释进行修改过来就好了

#### ---------------------------------------tips:  ----------------------------------------------------------------------------------------------------------------------------------------

如何查看你的有线网卡的编号呢？一般百度会告诉你`ifconfig`一下就行了。然而，新装的电脑上是没有安装这个工具的。所以你可以尝试输入以下命令查看网卡编号
``` bash
$ ip link
```

输出如下所示：
``` bash
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: enp0s31f6: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc fq_codel state DOWN mode DEFAULT group default qlen 1000
    link/ether 8c:16:45:28:8e:c5 brd ff:ff:ff:ff:ff:ff
3: wlp61s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP mode DORMANT group default qlen 1000
    link/ether d4:6d:6d:2b:4d:cd brd ff:ff:ff:ff:ff:ff

```
`lo`是内部环路，`enp0××××`是有线网卡，也就是我们查询的目标编号，`wl×××`是无线网卡的编号


#### ------------------------------------------------tips end ------------------------------------------------------------------------------------------------------------------------------------------------

修改了网络配置之后，执行一下命令重启一下系统的网络服务，然后就完事了～

``` bash
$ sudo systemctl restart networking.service
```

### Ubuntu 18.04的配置

等等，如果你是在`Ubuntu18.04`上做完上面的操作之后，你会发现你还是不能上网。你的网卡的地址被改了，但是`ip`没有。因为`Ubuntu 18.04`引入了新的网络管理工具`Netplan`。好吧，除了修改上面的那个文件之外，还需该修改`Netplan`的配置文件
``` bash
$ sudo gedit /etc/netplan/01-network-manager-all.yaml
```  

默认的内容是：
``` python
# Let NetworkManager manage all devices on this system                          
network:
  version: 2
  renderer: NetworkManager
```

按照上面的意思修改成自己对应的文件IP地址和网卡地址，在此不再添加注释：
``` python
network:
  ethernets:
    enp0s31f6:
      dhcp4: no
      addresses: [172.18.166.***/23]
      macaddress: 8c:ec:4b:45:**:**
      gateway4: 172.18.167.254
      nameservers:
        addresses: [10.8.8.8]
      optional: true
  version: 2

```

做完了上面的修改之后



``` bash
$ sudo netplan apply
```
然后就可以上网了。
上面两个配合文件，只修改一个，是不行的，亲测～

什么？你还是上不了网( ⊙ o ⊙ )啊！IP地址和Mac检查过了？配置都对了？    
那要不就重启一下电脑吧～如若重启电脑都解决不了，我也没有办法了,或许可以重装系统试试 ：）